In [2]:
import torch
from torch import nn

In [3]:
# Get data from the dataset

with open('../data/russian-novels/WAP.txt', 'r', encoding='utf-8-sig') as file:
    text = file.read()
    
chars = sorted(list(set(text)))
ctoi = {c: i+1 for i, c in enumerate(chars)}
ctoi['_'] = 0
itoc = {i: c for c, i in ctoi.items()}

encode = lambda x: torch.tensor([ctoi[c] for c in x], dtype=torch.long)
decode = lambda x: ''.join([itoc[i.item()] for i in x])

In [4]:
# Build dataset
block_size = 8

X, Y = [], []
X_test, Y_test = [], []

test = text[int(len(text) * 0.8):]
text = text[:int(len(text) * 0.8)]

for i in range(0, len(text) - block_size):
    X.append([ctoi[c] for c in text[i:i+block_size]])
    Y.append(ctoi[text[i+block_size]])

for i in range(0, len(test) - block_size):
    X_test.append([ctoi[c] for c in test[i:i+block_size]])
    Y_test.append(ctoi[test[i+block_size]])

X = torch.tensor(X)
Y = torch.tensor(Y)

X_test = torch.tensor(X_test)
Y_test = torch.tensor(Y_test)

X.shape, Y.shape

(torch.Size([2566610, 8]), torch.Size([2566610]))

In [5]:
def get_batch(batch_size=32):
    idx = torch.randint(0, X.size(0), (batch_size,))
    return X[idx], Y[idx]

def get_test_batch(batch_size=32):
    idx = torch.randint(0, X_test.size(0), (batch_size,))
    return X_test[idx], Y_test[idx]

batch = get_batch(32)

In [6]:

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.C = torch.randn(len(chars)+1, 2)
        self.W1 = torch.randn(block_size * 2, 200) * 0.2
        self.b1 = torch.randn(200) * 0.01
        self.W2 = torch.randn(200, len(chars)+1) * 0.01
        self.b2 = torch.randn(len(chars)+1) * 0
        self.parameters = [self.C, self.W1, self.b1, self.W2, self.b2] 
       
        
    def forward(self, x, y):
        
        emb = self.C[x].view(-1, 2*block_size)
        h = torch.relu(emb @ self.W1 + self.b1)
        logits = h @ self.W2 + self.b2
        preds = torch.softmax(logits, dim=1)
        loss = torch.nn.functional.cross_entropy(logits, y)
        
        return preds, loss

model = Model()

for p in model.parameters:
    p.requires_grad = True

In [7]:
x_train, y_train = batch

optimizer = torch.optim.SGD(model.parameters, lr=0.001, momentum=0.9)

for i in range(99999):
    
    optimizer.zero_grad()
    x, y = get_batch(32)
    
    outputs, loss = model(x, y)
    
    loss.backward()
    
    optimizer.step()
    
    # if i % 1000 == 0:
print(f'Loss: {loss.item()}')
        

Loss: 2.501145362854004


In [ ]:
# Sample the model

result = encode("War and ")
print("War and ", end='')
while result[-1] != 0:
    x = result[-8:]
    x = x.view(1, -1)
    y = torch.tensor([result[-1]])
    preds, loss = model.forward(x, y)
    next_char = torch.multinomial(preds[0], 1)
    result = torch.cat([result, next_char])
    print(decode(next_char) , end='')

In [366]:
# Test the model

x, y = get_test_batch(32)
outputs, loss = model(x, y)
print(loss.item())


2.2150869369506836


In [284]:
batch[0].shape

torch.Size([5, 3])